# Chapter 1: SMPC Example (Deprecated)

| Chapter  | Colab   | Kaggle          | Gradient      | Studio Lab             | Binder             |
|:---------|:--------|:----------------|:--------------|:-----------------------|:-------------------|
| [Chapter 1: SMPC Example](1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb)                         | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb)       | [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb)       | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb)       | [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb)       | [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/matthew-mcateer/practicing_trustworthy_machine_learning/HEAD?urlpath=https%3A%2F%2Fgithub.com%2Fmatthew-mcateer%2Fpracticing_trustworthy_machine_learning%2Fblob%2Fmain%2F1_privacy%2Fdeprecated%2FChapter_1_SMPC_Example_DEPRECATED.ipynb)              |

<!--
Originally found on GitHub at https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb
-->


The following is heavily based on an OpenMined tutorial written by [Ayoub Benaissa - Twitter: @y0uben11](https://twitter.com/y0uben11)

```text
This tutorial is currently optimized for running in a Docker container on a local machine.

The tutorial that inspired this previously could run on multiple Google Colab instances.

However, after Colab changed the accessibility of multiple runtimes, this is no longer the case.

Specific instructions for the new Colab version will be added in the future.
```

To run this example, clone the [OpenMined/Pygrid](https://github.com/OpenMined/PyGrid/commit/048be4ac52a53b5ca947b920e41de9a9e76a532a) repo and start the two GridNodes by running

```bash
cd
cd apps/node
./run.sh --id ID --port 3000 --start_local_db
```

and

```bash
cd
cd apps/node
./run.sh --id ID --port 3001 --start_local_db
```

in separate terminals.

Alternatively, you can use the colab notebooks

- [SMPC_Mock_Node_1.ipynb](1_privacy/deprecated/Chapter_1_SMPC_Mock_Node_1.ipynb)
- [SMPC_Mock_Node_2.ipynb](1_privacy/deprecated/Chapter_1_SMPC_Mock_Node_2.ipynb)

In [ ]:
#@title PySyft & PyTorch Setup { display-mode: "form" }
!pip -qq uninstall torch torchvision --y

try:
    import torch
except:
    print("PyTorch not installed. Installing...")
    !pip -qq install torch==1.8.0

    import torch
    print("Torch version: ", torch.__version__ )
    assert torch.__version__=='1.8.0', "torch version is not 1.8.0, the version needed to make this run correctly"

!pip -qq install syft==0.2.9
!pip -qq install crypten
!pip -qq install loguru
#!pip -qq install --upgrade --force-reinstall lz4
#!pip -qq install --upgrade --force-reinstall websocket
#!pip -qq install --upgrade --force-reinstall websockets

In [ ]:
!pip -qq install watermark
%load_ext watermark

In [ ]:
%watermark -a "Practicing Trustworthy machine Learning" -u -d -v -m -p syft

In [ ]:
%watermark -a "Practicing Trustworthy machine Learning" -u -d -v -m -p syft,crypten

In [ ]:
%watermark -a "Practicing Trustworthy machine Learning" -u -d -v -m -p syft,crypten,loguru,torch,torchvision

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import crypten
import syft
from time import time
from syft.frameworks.crypten.context import run_multiworkers
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient
     

In [ ]:
!wget "https://raw.githubusercontent.com/facebookresearch/CrypTen/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py" -O "mnist_utils.py"
     

In [ ]:

!python "mnist_utils.py" --option features --reduced 100 --binary
     

In [ ]:
torch.manual_seed(0)
torch.set_num_threads(1)
hook = syft.TorchHook(torch)

#from syft.workers.node_client import NodeClient # No module named 'syft.workers.node_client'

In [ ]:


# Define an example network
class ExampleNet(nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 16 * 12 * 12)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out
     

In [ ]:

# Syft workers
print("[%] Connecting to workers ...")
ALICE = DataCentricFLClient(hook, "ws://localhost:3000")
BOB = DataCentricFLClient(hook, "ws://localhost:3001")
print("[+] Connected to workers")

print("[%] Sending labels and training data ...")
# Prepare and send labels
label_eye = torch.eye(2)
labels = torch.load("/tmp/train_labels.pth")
labels = labels.long()
labels_one_hot = label_eye[labels]
labels_one_hot.tag("labels")
al_ptr = labels_one_hot.send(ALICE)
bl_ptr = labels_one_hot.send(BOB)

# Prepare and send training data
alice_train = torch.load("/tmp/alice_train.pth").tag("alice_train")
at_ptr = alice_train.send(ALICE)
bob_train = torch.load("/tmp/bob_train.pth").tag("bob_train")
bt_ptr = bob_train.send(BOB)

print("[+] Data ready")
     

In [ ]:

# Initialize model
dummy_input = torch.empty(1, 1, 28, 28)
pytorch_model = ExampleNet()
     

In [ ]:

@run_multiworkers([ALICE, BOB], master_addr="127.0.0.1", model=pytorch_model, dummy_input=dummy_input)
def run_encrypted_training():
    rank = crypten.communicator.get().get_rank()
    # Load the labels
    worker = syft.frameworks.crypten.get_worker_from_rank(rank)
    labels_one_hot = worker.search("labels")[0]
    # Load data:
    x_alice_enc = crypten.load("alice_train", 0)
    x_bob_enc = crypten.load("bob_train", 1)
    # Combine the feature sets: identical to Tutorial 3
    x_combined_enc = crypten.cat([x_alice_enc, x_bob_enc], dim=2)
    # Reshape to match the network architecture
    x_combined_enc = x_combined_enc.unsqueeze(1)
    # model is sent from the master worker
    model.encrypt()
    # Set train mode
    model.train()
    # Define a loss function
    loss = crypten.nn.MSELoss()
    # Define training parameters
    learning_rate = 0.001
    num_epochs = 2
    batch_size = 10
    num_batches = x_combined_enc.size(0) // batch_size

    for i in range(num_epochs):
        # Print once for readability
        if rank == 0:
            print(f"Epoch {i} in progress:")
            pass

        for batch in range(num_batches):
            # define the start and end of the training mini-batch
            start, end = batch * batch_size, (batch + 1) * batch_size
            # construct AutogradCrypTensors out of training examples / labels
            x_train = x_combined_enc[start:end]
            y_batch = labels_one_hot[start:end]
            y_train = crypten.cryptensor(y_batch, requires_grad=True)
            # perform forward pass:
            output = model(x_train)
            loss_value = loss(output, y_train)
            # set gradients to "zero"
            model.zero_grad()
            # perform backward pass:
            loss_value.backward()
            # update parameters
            model.update_parameters(learning_rate)
            # Print progress every batch:
            batch_loss = loss_value.get_plain_text()
            if rank == 0:
                print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")

    model.decrypt()
    # printed contain all the printed strings during training
    return printed, model
     


In [ ]:

print("[%] Starting computation")
func_ts = time()
result = run_encrypted_training()
func_te = time()
print(f"[+] run_encrypted_training() took {int(func_te - func_ts)}s")
printed = result[0][0]
model = result[0][1]
print(printed)
     

In [ ]:

cp = syft.VirtualWorker(hook=hook, id="cp")
model.fix_prec()
model.share(ALICE, BOB, crypto_provider=cp)
print(model)
print(list(model.parameters())[0])